# MCP Client in Jupyter Notebook
This notebook demonstrates how to connect to an MCP math server and use LangChain tools.

In [3]:
import sys
import asyncio
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent

async def run_mcp(chat_history=None):
    math_params = StdioServerParameters(
        command=sys.executable,
        args=["d:\\MAT496-Monsoon2025\\mod04_MCP\\servers\\math_server.py"],
    )
    fetch_params = StdioServerParameters(
        command=sys.executable,
        args=["-m", "mcp_server_fetch"],  # assumes installed via pip
    )

    async with stdio_client(math_params, errlog=None) as (math_read, math_write), \
               stdio_client(fetch_params, errlog=None) as (fetch_read, fetch_write):
        async with ClientSession(math_read, math_write) as math_session, \
                   ClientSession(fetch_read, fetch_write) as fetch_session:
            await math_session.initialize()
            await fetch_session.initialize()
            math_tools = await load_mcp_tools(math_session)
            fetch_tools = await load_mcp_tools(fetch_session)
            tools = math_tools + fetch_tools
            agent = create_react_agent("groq:openai/gpt-oss-120b", tools)
            result = await agent.ainvoke(chat_history)
            return result

In [4]:
chat_history = {"messages": [{"role": "user", "content": "hi?"}]}

chat_history = await run_mcp(chat_history)
chat_history

{'messages': [HumanMessage(content='hi?', additional_kwargs={}, response_metadata={}, id='98d7a5cd-9ac3-4607-a4f0-3bda4610d3cd'),
  AIMessage(content='Hello! How can I help you today?', additional_kwargs={'reasoning_content': 'The user says "hi?" It\'s a simple greeting. I should respond politely.'}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 302, 'total_tokens': 337, 'completion_time': 0.069621966, 'prompt_time': 0.011618035, 'queue_time': 0.051278275, 'total_time': 0.081240001}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_ed9190d8b7', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--795a9859-91d7-4a49-b355-793cdb180473-0', usage_metadata={'input_tokens': 302, 'output_tokens': 35, 'total_tokens': 337})]}

In [5]:
new_message = "Go the the site https://krajit.github.io/ and find Ajit's email address"

chat_history["messages"].append({"role": "user", "content": new_message})
chat_history = await run_mcp(chat_history)
chat_history

{'messages': [HumanMessage(content='hi?', additional_kwargs={}, response_metadata={}, id='98d7a5cd-9ac3-4607-a4f0-3bda4610d3cd'),
  AIMessage(content='Hello! How can I help you today?', additional_kwargs={'reasoning_content': 'The user says "hi?" It\'s a simple greeting. I should respond politely.'}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 302, 'total_tokens': 337, 'completion_time': 0.069621966, 'prompt_time': 0.011618035, 'queue_time': 0.051278275, 'total_time': 0.081240001}, 'model_name': 'openai/gpt-oss-120b', 'system_fingerprint': 'fp_ed9190d8b7', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--795a9859-91d7-4a49-b355-793cdb180473-0', usage_metadata={'input_tokens': 302, 'output_tokens': 35, 'total_tokens': 337}),
  HumanMessage(content="Go the the site https://krajit.github.io/ and find Ajit's email address", additional_kwargs={}, response_metadata={}, id='978ad4e1-e0f8-4431-bd3c-3620fec5d8c5'),
  AIMessage(co

Note: We will learn how to create react_agents when we will discuss langgraph